In [19]:
!pip install datasets transformers evaluate

In [44]:
!pip install transformers[torch]

In [46]:
pip install scikit-learn


   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.2 MB 1.3 MB/s eta 0:00:08
    --------------------------------------- 0.2/9.2 MB 1.8 MB/s eta 0:00:05
   - -------------------------------------- 0.3/9.2 MB 2.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/9.2 MB 2.6 MB/s eta 0:00:04
   --- ------------------------------------ 0.7/9.2 MB 3.1 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.2 MB 3.8 MB/s eta 0:00:03
   ------ --------------------------------- 1.4/9.2 MB 4.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.5/9.2 MB 4.4 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/9.2 MB 3.8 MB/s eta 0:00:02
   -------- ------------------------------- 2.0/9.2 MB 4.3 MB/s eta 0:00:02
   ---------- ----------------------------- 2.4/9.2 MB 4.7 MB/s eta 0:00:02
   ----------- ---------------------------- 2.6/9.2 MB 4.7 MB/s eta 0:00:02
   ----------- ----

In [ ]:
pip install sklearn

In [20]:
import numpy as np
import pandas as pd

In [21]:
train_set = pd.read_csv("./data/kaggle/Train.csv")
train_set.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [22]:
test_set = pd.read_csv("./data/kaggle/Test.csv")
test_set.head()

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0


In [23]:
valid_set = pd.read_csv("./data/kaggle/Valid.csv")
valid_set.head()

,text,label
0,It's been about 14 years since Sharon Stone aw...,0
1,someone needed to make a car payment... this i...,0
2,The Guidelines state that a comment must conta...,0
3,This movie is a muddled mish-mash of clichés f...,0
4,Before Stan Laurel became the smaller half of ...,0


In [24]:
def to_lower(text):
    text = text.lower()
    return text

In [25]:
train_set["text"] = train_set["text"].map(to_lower)
test_set["text"] = test_set["text"].map(to_lower)
valid_set["text"] = valid_set["text"].map(to_lower)

In [26]:
train_set.drop_duplicates(inplace = True)

In [27]:
from datasets import Dataset

train_set = Dataset.from_pandas(train_set)
test_set = Dataset.from_pandas(test_set)
valid_set = Dataset.from_pandas(valid_set)

In [28]:
train_set, test_set, valid_set

(Dataset({
     features: ['text', 'label', '__index_level_0__'],
     num_rows: 39723
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 5000
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 5000
 }))

In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [30]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train_tokenized = train_set.map(tokenize_function, batched=True)
valid_tokenized = valid_set.map(tokenize_function, batched=True)

Map:   0%|          | 0/39723 [00:00<?, ? examples/s]

Map: 100%|██████████| 5000/5000 [00:03<00:00, 1577.33 examples/s]


In [31]:
train_tokenized.shuffle(seed=42)

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 39723
})

In [32]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
import torch
from torch.nn.parallel import DataParallel

# Move model to GPU(s)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if torch.cuda.device_count() > 1:
    model = DataParallel(model)

In [33]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [34]:
import evaluate

metric = evaluate.load("accuracy")

ImportError: To be able to use evaluate-metric/accuracy, you need to install the following dependencies['scikit-learn'] using 'pip install sklearn' for instance'

In [35]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [36]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",save_steps=3000,
                                  num_train_epochs= 2)

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=valid_tokenized,
    compute_metrics=compute_metrics
)

In [38]:
trainer.train()

  0%|          | 30/9932 [23:42<137:37:09, 50.03s/it]

KeyboardInterrupt: 

In [ ]:
test_tokenized = test_set.map(tokenize_function, batched=True)

In [ ]:
trainer.evaluate()

In [ ]:
model.save_pretrained("./sentiment")

In [ ]:
tokenizer.save_pretrained("./sentiment")

In [ ]:
!zip -r bert-sentiment.zip sentiment